In [ ]:
# Importing standard python libraries
import numpy as np
from math import pi,sqrt
import matplotlib.pyplot as plt

# Importing standard Qiskit libraries
from qiskit import QuantumCircuit, transpile, Aer, IBMQ, execute, QuantumRegister
from qiskit.tools.jupyter import *
from qiskit.visualization import * # plot_bloch_multivector
from ibm_quantum_widgets import * # CircuitComposer
from qiskit.providers.aer import QasmSimulator
from qiskit.quantum_info import Statevector

# Loading your IBM Quantum account(s)
provider = IBMQ.load_account()

In [ ]:
# Start with an one qubit quantum circuit yielding a nice fractal. Change the circuit as you like.
circuit = QuantumCircuit(1,1)
circuit.h(0)
circuit.u(pi/4, -pi/3, pi/8, 0)
editor = CircuitComposer(circuit=circuit)
editor

In [ ]:
# View the circiut quantum state on the Bloch sphere 
qc1 = editor.circuit
plot_bloch_multivector(qc1)

In [ ]:
qc1.draw()

In [ ]:
# Run the circuit with the state vector simulator to obtain a noise-free fractal.
backend = Aer.get_backend('statevector_simulator')
out = execute(qc1,backend).result().get_statevector()
print(out)

# Extract the first element of the state vector as z0 and the second element as z1.
z0 = out.data[0]
z1 = out.data[1]

# Goal: One complex number for the Julia set fractal. 
if z1.real != 0 or z1.imag != 0:
    z = z0/z1
    z = round(z.real, 2) + round(z.imag, 2) * 1j
    print("z0/z1= ",z)
else:
     z = 0 

print("z= ",z)

# Define the size
size = 1000
heightsize = size
widthsize = size


def julia_set(c=z, height=heightsize, width=widthsize, x=0, y=0, zoom=5, max_iterations=70):

    # To make navigation easier we calculate these values
    x_width = 1.5
    y_height = 1.5*height/width
    x_from = x - x_width/zoom
    x_to = x + x_width/zoom
    y_from = y - y_height/zoom
    y_to = y + y_height/zoom
    
    # Here the actual algorithm starts and the z paramter is defined for the Julia set function
    x = np.linspace(x_from, x_to, width).reshape((1, width))
    y = np.linspace(y_from, y_to, height).reshape((height, 1))
    z = x + 1j * y
    
    # Initialize c to the complex number obtained from the quantum circuit
    c = np.full(z.shape, c)
    
    # To keep track in which iteration the point diverged
    div_time = np.zeros(z.shape, dtype=int)
    
    # To keep track on which points did not converge so far
    m = np.full(c.shape, True, dtype=bool)
    
    for i in range(max_iterations):
        z[m] = z[m]**2 + c[m] 
        m[np.abs(z) > 2] = False
        div_time[m] = i
    return div_time


# plot the Julia set fractal
plt.imshow(julia_set(), cmap='magma') # viridis', 'plasma', 'inferno', 'magma', 'cividis'
plt.show()

# Running on a real quantum computer
### Identify the least busy system, perform measurements and run on the backend

In [ ]:
from qiskit.providers.aer import noise
from qiskit_aer.noise import NoiseModel

IBMQ.load_account()
provider = IBMQ.get_provider(hub = 'ibm-q')

# or, specifically choose a device. Paris performs the best out of any device so far
device = provider.get_backend('ibm_oslo')

# Get the noise from quantic
noise_model = NoiseModel.from_backend(device)
# Get coupling map from backend
coupling_map = backend.configuration().coupling_map
# Get basis gates from noise model
basis_gates = noise_model.basis_gates



In [ ]:
# Add measurement - used to calculate the modified state vector
qc1.measure(0, 0)
qc1.draw()

In [ ]:
shotno=1024
job = execute(qc1, Aer.get_backend('qasm_simulator'),
                 coupling_map=coupling_map,
                 basis_gates=basis_gates,
                 noise_model=noise_model,
                 shots=shotno)

# Get the results from the computation
result = job.result()

## Perform calculations and plot result

In [ ]:
# Get count measurement results
counts = result.get_counts()


prob0qc = counts['0']/shotno
prob1qc = counts['1']/shotno
#print("Observed probabilities of measuring the computational basis states", round(prob0qc,3), round(prob1qc,3))

prob0statevec = out.probabilities()[0]
prob1statevec = out.probabilities()[1]

# one simple approach to calculate a quantum computer-modified complex number # amplitude^2 = probability
z0qc = z0*sqrt(prob0qc/prob0statevec)
z1qc = z1*sqrt(prob1qc/prob1statevec)
zqc = z0qc/z1qc

# compare the simulator state vector with the-running-on-a-real-quantum-computer modified state vector
print(z,zqc)

# Plot both Julia set fractals for comparison - the simulator based on the left and the modified on the right 
f, axarr = plt.subplots(1,2,figsize=(12, 12))
axarr[0].imshow(julia_set(c=z), cmap='magma')
axarr[1].imshow(julia_set(c=zqc), cmap='gray')

# Save the subplot.
bbox = axarr[1].get_tightbbox(f.canvas.get_renderer())

In [ ]:
# Save the subplot.
import os

counter = 0
filename = "heightmap{}.png"
while os.path.isfile(filename.format(counter)):
    counter += 1
filename = filename.format(counter)

# Save just the portion _inside_ the second axis's boundaries
extent = axarr[1].get_window_extent().transformed(f.dpi_scale_trans.inverted())
f.savefig('heightmaps/quantum/' + filename, bbox_inches=extent,dpi=300)

#blender heightmap
extent = axarr[1].get_window_extent().transformed(f.dpi_scale_trans.inverted())
f.savefig('heightmap', bbox_inches=extent,dpi=300)

#no quantum computer heightmap
extent = axarr[0].get_window_extent().transformed(f.dpi_scale_trans.inverted())
f.savefig('heightmaps/normal/ax1_figure.png', bbox_inches=extent,dpi=300)

## Run blender script to automatically generate a 3D model of the fractal terrain

In [ ]:
import subprocess

PATH_TO_BLENDER = "Blender app path"
PATH_TO_BLENDER_SCRIPT = "Blender script path"
PATH_TO_BLENDER_SCENE = "./blender_script.py"

# Path to the Blender executable
blender_executable = PATH_TO_BLENDER

# Path to the Blender scene file
scene_file = PATH_TO_BLENDER_SCENE

# Path to the Blender script file
script_file = PATH_TO_BLENDER_SCRIPT



# Run the Blender script to set up the scene
subprocess.call([
    blender_executable,
    scene_file,
    '-b',  # Run in background mode
    '-P', script_file  # Run the script file
])
